In [1]:
import sys
sys.path.append('C:/Users/Kai/Desktop/vis_scBERT/DNABERT_2/')
from get_gene_name_and_seq import *

# Get NT sequence from serval patheways of cmiu

In [4]:
raw='''
00190  Oxidative phosphorylation
00680  Methane metabolism
00910  Nitrogen metabolism
00920  Sulfur metabolism
'''
pathway_list = clean_raw_pathway_str(raw, 'cmiu')
cmiu_dic = pathway2NTseq_cmiu(pathway_list)

In [8]:
for k,v in cmiu_dic.items():
    print(k, len(v))

cmiu00190 12
cmiu00680 28
cmiu00910 7
cmiu00920 6


# Build data for "gene sense" task

In [8]:
raw='''
cmiu00010  	Glycolysis / Gluconeogenesis
cmiu00051  	Fructose and mannose metabolism
cmiu00052  	Galactose metabolism
cmiu00500  	Starch and sucrose metabolism
cmiu00520  	Amino sugar and nucleotide sugar metabolism
cmiu00521  	Streptomycin biosynthesis
cmiu01100  	Metabolic pathways
cmiu01110  	Biosynthesis of secondary metabolites
cmiu01120  	Microbial metabolism in diverse environments
cmiu01200  	Carbon metabolism
cmiu01250  	Biosynthesis of nucleotide sugars
'''
pathway_list = clean_raw_pathway_str(raw, 'cmiu')
cmiu_dic = pathway2NTseq_cmiu(pathway_list)

Getting NT sequences: 100%|██████████| 11/11 [03:19<00:00, 18.17s/it]


In [9]:
cmiu_dic

{'cmiu00010': {'cmiu:B1H56_02580': 'ATGAAAGGAACGCATATGCTGACTGCAAACGAGTTTATCAGGAAACATAAAATGGAATACACGTCCATTGACGTAAAAAACGAGATGGACACTTACATCGATGAAATGAAAAAGGGATTGTGCGGCGAGAAGAGTTCGCTGCTGATGATCCCTTCCTATATTACGCTGAAAAGCGAAGTGAAGCGGGAAAAACCGGTGATCTGCGTAGACGCGGGCGGGACGAATCTGCGCGTTGCGGTGGCGAAATTTTCCGAGGACGGCTCTTTTTGCACGGAGGAATTCCAGCGGTATTTGATGCCGGGTGTGGAAAAGGAACTTGAGGCCGAAGAATTTTTTGATATTTTGGCGGATTATATCCTGCCTTTTACGAAAATCACGAAAAATATTGTGATTTCATTCGCTTACCGGGCAAAAATACTGCCCGATATCGACTGCGAGATCGTAGAAATCACCAAAGAAGTCAAAGTGAAAAATGCAGGCGGAAAACTGCTGGGCAGAGAGATTTGCGCGGCCTTGGCGAAGAAGGGCGCGGACGGCTGTAATATCGTTGTGGTAAACGACTCCGTGGCAACGGCGCTTGCCGGAAAAGCGGAAAAACTGAACGACGGATACGGCTCCTTTACGGGGACGATACTTGGTACGGGAAGCAACAGCTGCTATATTGAATATATGCGCAACATCGCGAAGCTTGAAGGCGGCGAAGAGGGCCTGATGGTGATTAATACGGAGGCGGGCAGCTATAATAAGGTGCCGCGCAGCGATATCGATATTGCCTATGACGAAAGCACACAAAACCCTGGGATCGGGGTGTTTGAGAAAATGACCAGCGGCGGCTACTTGGGCCCCCTGTGTGATTTTACCCTGCGCACGGCGGCGCGGGAAGGTGTCTTTGAAAAAGGATTTTCCATGGCAGGGACGGTCTCGACCGTCGACGTGAACGCTTTCCTGACGGACGGGTCGGGCG